In [32]:
import tweepy
import webbrowser
import re
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load('en_core_web_lg')

In [8]:
import keys as keys

In [11]:
auth = tweepy.OAuthHandler(consumer_key=keys.CONSUMER_KEY, consumer_secret=keys.CONSUMER_SECRET)

In [12]:
auth.set_access_token(keys.ACCESS_KEY, keys.ACCESS_SECRET)

In [13]:
api = tweepy.API(auth)

In [14]:
# Keytword/hashtag to search about
search_item = 'xbox'
num_of_tweet_search = 50

In [ ]:
tweets = tweepy.Cursor(api.search, q=search_item, lang='English').items(num_of_tweet_search)

In [16]:
from textblob import TextBlob

In [17]:
a = TextBlob('This new xbox is just expensive and adds nothing new.')

In [18]:
a.sentiment.polarity

-0.07575757575757576

In [20]:
import src.useful_codes as uc

In /opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/learn-env

In [23]:
model = uc.load_data('models/binary_word_emb')

In [24]:
a = 'This new xbox is just expensive and adds nothing new.'

In [37]:
binary_word_emb = uc.load_data('models/binary_word_emb')

def prototype(model='binary'):
    user_input = input('Write a review: ')
    
    text = get_clean_text(user_input)
    word_emb = get_vector(text)
    if model=='binary':
        model = binary_word_emb
    else:
        model = multiclass_word_emb
    print('-'*30)
#     print('Product: ', predict_product(text))
    print('Sentiment: ', predict_sentiment(model, word_emb))
    print('Confidence: ', round(get_confidence(model, word_emb)*100, 2), '%')
    print('-'*30)
    print('\n')
def get_clean_text(text):
    text = ' '.join(clean_text(text))
    return text

def get_vector(text):
    word_emb = np.concatenate(get_vec(text).reshape(-1,1), axis=0).reshape(-1, 300)
    return word_emb

def predict_sentiment(model, word_emb):
    predict = model.predict(word_emb)
    return predict[0]

def predict_product(text):
    predict = product_predictor.predict([text])
    return predict[0]

def get_confidence(model, word_emb):
    prob = model.predict_proba(word_emb)
    return max(prob[0])

In [ ]:
while True:
    prototype()

Write a review: I am the worst
------------------------------
Sentiment:  Negative
Confidence:  99.94 %
------------------------------


Write a review: Why is this new xbox so slow?
------------------------------
Sentiment:  Negative
Confidence:  81.25 %
------------------------------


Write a review: why is this new xbox so awesome?
------------------------------
Sentiment:  Positive
Confidence:  98.67 %
------------------------------


Write a review: new xbox controllers are dope!
------------------------------
Sentiment:  Positive
Confidence:  82.6 %
------------------------------


Write a review: new xbox controllers
------------------------------
Sentiment:  Positive
Confidence:  79.91 %
------------------------------


Write a review: im dope!
------------------------------
Sentiment:  Positive
Confidence:  88.99 %
------------------------------


Write a review: i
------------------------------
Sentiment:  Positive
Confidence:  69.12 %
------------------------------


Write 

In [34]:
def get_vec(x):
    """
    Extracts vector out of string
    Args:
        x (str): string
    Returns:
        vec (array): word vector
    """

    doc = nlp(x)
    vec = doc.vector
    return vec


In [28]:
# spaCy tokenizer
def clean_text(text, stopwords=False, tweet=True):
    """
    Cleans and tokenizes tweet text data.
    Args:
        text (str): tweet text data
        
        stopwords (bool): True if stopwords needs to be removed
        
        tweet (bool): True if text data are tweets.
    
    Returns:
        tokens (array): Array of tokenized words from given text.
    """
    if tweet:
        text = re.sub(r'@\S+', '', text) # Gets rid of any mentions
        text = re.sub(r'RT\S+', '', text) # Gets rid of any retweets
        text = re.sub(r'#', '', text) # Gets rid of hashtag sign
        text = re.sub(r'https?:\/\/\S+', '', text) # Gets rid of any links
        text = re.sub(r'[0-9]+.?[0-9]+', '', text) # Gets rid of X.X where X are numbers
        text = re.sub(r'#?(sx|Sx|SX)\S+', '', text) # Gets rid common mentions
        text = re.sub(r'(&quot;|&Quot;)', '', text) # Gets rid of quotes
        text = re.sub(r'(&amp;|&Amp;)', '', text) # Gets rid of quotes
        text = re.sub(r'link', '', text) # Gets rid of quotes
    doc = nlp(text)

    tokens = []
    for token in doc:
        if token.lemma_ != '-PRON-': # if token is not a pronoun
            temp_token = token.lemma_.lower().strip()
        else:
            temp_token = token.lower_
        tokens.append(temp_token)
    
    if stopwords:
        tokens_stopped = [token for token in tokens if token not in stopwords_list and len(token)>2]
    else:
        tokens_stopped = [token for token in tokens if len(token)>2]
    
    return tokens_stopped
